### Imports

In [3]:
import numpy as np
import pandas as pd
from pyproj import Transformer
from datetime import date,datetime
#from bidict import bidict

In [4]:
#Import general library
import bokeh

#Import to handle data parsed to bokeh plots
from bokeh.models import ColumnDataSource

#Import to handle bokeh notebook
from bokeh.io import output_notebook

#Import for the background of the map plot
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TONER, WIKIMEDIA

#Import to create plot, show plot, and output plot
from bokeh.plotting import figure, output_file, show

#Import to handle layout of the bokeh plots
from bokeh.layouts import column, row, layout

#Importing the RangeTool used in the Timeline plot
from bokeh.models import RangeTool

#Import to set the range of the Timeline
from bokeh.models import Range1d

#Import for the button and datepicker widget
from bokeh.models import Button, DatePicker

#Import for callback on an event
from bokeh.events import ButtonClick

#Import palette for the color mapper used in geo_map
#Possible other color palettes
# https://colorcet.holoviz.org/user_guide/index.html
import colorcet as cc
#palette = cc.CET_L9

#Import palette for the histogram colors
from bokeh.palettes import OrRd3, Viridis256

palette = Viridis256


#Import log_cmap to support the color mapper for the geo_map
from bokeh.transform import log_cmap, linear_cmap

#Importing the select tools for bokeh plots
from bokeh.models import LassoSelectTool, BoxSelectTool

#Import the tools for the color bar
from bokeh.models import ColorBar, LogTicker
#Other option for ticker: AdaptiveTicker

#Import legend to show the legend for the timeline and histogram
from bokeh.models import Legend

#Import for the histogram model
from bokeh.models import FactorRange

#Import for the coloring of the bars in the histogram plot
from bokeh.transform import factor_cmap

#Import to show multiple tabs
from bokeh.models import Panel, Tabs

output_notebook()

Loading BokehJS ...

### Assumptions

In [5]:
#screen resolution
screen_height = 680
screen_length = 920

#Location of dataset csv
location_dataset = "C:/Users/DESKTOP-2/Documents/Courses/Master Project/Data/complete_swiss_dataset.csv"

### Data import and preparation

In [6]:
#Columns that we want to keep from the data we load in
import_columns = ['created_at_CET','latitude','longitude','Overall.score','Anger','Confusion',
                  'Disgust','Fear','Happiness','Sadness','Shame','Surprise']

#Load the data into the python environment
twitter_data = pd.read_csv(location_dataset,encoding = "ISO-8859-15",low_memory=False)

#Select only the columns that are relevant
#Timeline Map Histogram data (TMH_data)
TMH_data = twitter_data[import_columns].sort_values(by='created_at_CET',ignore_index=True)

#Round all entries in dataframe
#Mainly to remove specific gps locations of users
TMH_data = TMH_data.round(3)

#Format the string input to datetime
TMH_data['created_at_CET'] = pd.to_datetime(TMH_data['created_at_CET'], format='%Y-%m-%d %H:%M:%S')

#Obtain a list of tweets which are within the specified coordinates
geo_bounds = np.where((TMH_data['latitude'] >= 45.8) &
                      (TMH_data['latitude'] <= 47.9) &
                      (TMH_data['longitude'] >= 5.9) &
                      (TMH_data['longitude'] <= 10.6))[0]

#Set the data to only the points within the bounds
TMH_data = TMH_data.iloc[geo_bounds]

#Sort the data by the date at creation
TMH_data = TMH_data.sort_values(by='created_at_CET')

#Removing the entire dataset now that the selection has been made
twitter_data = None

In [7]:
#Declare transformer to convert from epsg4326 to epsg 3856 (web mercator)
transformer_merc = Transformer.from_crs("EPSG:4326", "EPSG:3857")

#Transform the coordinates from epsg 4326 to epsg 3857
mercator_transform = transformer_merc.transform(TMH_data['latitude'].values,
                                                TMH_data['longitude'].values)

#
TMH_data['x_mercator'] = mercator_transform[0].round(1)
TMH_data['y_mercator'] = mercator_transform[1].round(1)

#Declare transformer to convert from epsg4326 to epsg 21781 (swiss coordinate system)
transformer_swiss = Transformer.from_crs("EPSG:4326", "EPSG:21781")

#Transform the coordinates from epsg 4326 to epsg 3857
swiss_transform = transformer_swiss.transform(TMH_data['latitude'].values,
                                                 TMH_data['longitude'].values)

#
TMH_data['x_swiss'] = (swiss_transform[0] / 1e3).round()
TMH_data['y_swiss'] = (swiss_transform[1] / 1e3).round()

TMH_data['ID'] = TMH_data.index

In [8]:
#Create a dataframe that can be easily queried by time
time_table = TMH_data[['created_at_CET','ID']]
time_table.index = time_table['created_at_CET']
del time_table['created_at_CET']

In [9]:
# TABLE - timeline_data
# contains the amount of tweets per day and their corresponding tweet IDs

timeline_data = pd.DataFrame()

timeline_data['dates'] = TMH_data['created_at_CET'].copy(deep=True)

timeline_data['ID'] = timeline_data.index

timeline_data['hour_minute'] = timeline_data['dates'].dt.strftime('%H-%m')

timeline_data['dates'] = timeline_data['dates'].dt.strftime('%Y-%j')

del timeline_data['ID']

In [10]:
#Function to compute the number of tweets on days
#Input - list of tweet IDs

def tweets_per_day(IDs=None):
    #If no IDs have been passed then return the counts for all dates in the data
    #TODO: link timeline_data to the database table
    if(IDs == None):
        output = np.array(np.unique(timeline_data.iloc[:]['dates'],return_counts=True))
    else:
        output = np.array(np.unique(timeline_data.iloc[IDs]['dates'],return_counts=True))
    
    output[0] = pd.to_datetime(output[0],format='%Y-%j')
    
    return output
    
    

### Map bokeh plot

In [11]:
#Function to compute the map points and their densities given a certain timeframe
def compute_map_data(IDs = []):
    
    #Copy the relevant data from the dataframe
    if(len(IDs) == 0):
        #If no IDs have been parsed, then return all elements
        Map_data = TMH_data.iloc[:][['x_mercator','y_mercator']].sort_values(by=['x_mercator','y_mercator']).copy()
    else:
        Map_data = TMH_data.iloc[IDs][['x_mercator','y_mercator']].sort_values(by=['x_mercator','y_mercator']).copy()

    #Reset the index for the calculation of densities
    Map_data = Map_data.set_index(np.arange(0,len(Map_data)))

    #Drop all the duplciates, which keeps the first occuring entry in the dataframe
    Map_data = Map_data.drop_duplicates(subset=['x_mercator','y_mercator'])

    #Declare an array the length of the coordinates without duplicates
    #Will represent the density at each coordinate
    density = np.ones(len(Map_data))

    #Loop the array with all the coordinates
    for i in range(0,len(Map_data)-1):
        density[i] = Map_data.index[i+1] - Map_data.index[i]

    #Add the column to the Map_data
    Map_data['density'] = density
    
    return Map_data

### Histogram bokeh plot

In [12]:
#Function to find the values for a bar on a plot, given the IDs parsed
def compute_vbar_hist(IDs_selected = [],IDs_total = [], column_list = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']):
    
    if(len(IDs_selected) == 0):
        data_selected = TMH_data[column_list]
    else:
        data_selected = TMH_data.iloc[IDs_selected][column_list]
    
    if(len(IDs_total) == 0):
        data_total = TMH_data[column_list]
    else:
        data_total = TMH_data.iloc[IDs_total][column_list]
        
    mean_values = np.ones(len(column_list)*2)*-1
    
    #Loop all the columns in column_list
    for i in range(0,len(column_list)):
        mean_values[i*2] = data_selected[column_list[i]].mean()
        mean_values[(i*2)+1] = data_total[column_list[i]].mean()
        
    x = [(column,data_type) for column in column_list for data_type in ["Selected","Total"]]
    
    output = pd.DataFrame()
    
    output['measure'] = x
    output['mean'] = mean_values
    
    return output

In [13]:
#Function to compute the differences between the selected and the total
def measure_diff_hist(df,measures = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']):
    
    output_diff = np.zeros(int(len(df)/2))

    for i in range(0,int(len(df)/2)):
        output_diff[i] = ((df.iloc[i*2]['mean'] / df.iloc[(i*2)+1]['mean'])-1)*100
        
    output = pd.DataFrame()
    
    output['measure'] = measures
    output['difference'] = output_diff
    
    return output

### Adding all the bokeh plots together

In [14]:
def TMH_app(doc):
    
    ###  Timeline (START) ###
    
    #Create the Timeline figure
    Timeline = figure(background_fill_color="#fafafa",
                      x_axis_type='datetime',
                      x_axis_label="Date",
                      #y_axis_type="log",
                      y_axis_label="Amount of tweets",
                      tools="pan,box_zoom,wheel_zoom,reset",
                      toolbar_location="above",
                      plot_width=int(screen_length*0.8),
                      plot_height=int(screen_height*0.3))
    
    total_tweet_counts = tweets_per_day()
    
    #Load the data for the timeline into a ColumnDataSource for the plot
    source_total = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                              freq=total_tweet_counts[1]))
    
    #Declare an empty list to contain the items in the legend
    legend_items = []
    
    #Plot a line on the graph for the total amount of tweets per day
    total_tweets_per_day = Timeline.line('date','freq',source=source_total,line_color="black")
    
    #Add the total tweets to the legend
    legend_items.append(("Total",[total_tweets_per_day]))
    
    #Load the data for the geo-selected line, at initialization its 0 for all dates
    source_selected = ColumnDataSource(data=dict(date=total_tweet_counts[0],
                                                 freq=np.zeros(len(total_tweet_counts[1]))))
    
    #Plot the line for the selected tweets
    selected_tweets_per_day = Timeline.line('date','freq',source=source_selected,line_color="orange")
    
    #Add the selected tweets to the legend
    legend_items.append(("Geo selected",[selected_tweets_per_day]))
    
    #Set the selected line to not be visible initially
    selected_tweets_per_day.visible = False
    
    #Add the legend_items to the legend object
    legend = Legend(items = legend_items)
    
    #Set the onclick action of the legend to hide
    legend.click_policy="hide"
    
    Timeline.add_layout(legend,'left')

    
    #Declare a button which will update the values in Map according to the time period
    update_time_button = Button(label="Sync Time", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    #Callback to handle the changing x range of the range tool
    #Will update the corresponding start and end dates in the datepickers
    #datepickers: start_date_input and end_date_input
    def range_tool_to_datepicker_handler(attr,old,new):

        
        #If the starting value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.start) == float or
           type(range_tool.x_range.start) == int):
            #Extract the start value of the interval and convert it into correct format for datepicker
            new_start_date = datetime.fromtimestamp(range_tool.x_range.start / 1e3).date()
            
            #Check whether the date is less than the minimum of of the data
            if(new_start_date >= min_date_interval):
                
                #If the start date is later than the global minimum then set it as the new date
                start_date_input.value = new_start_date
            else:
                #If the date is before the global minimum then take the global minimum as the start of the range
                start_date_input.value = min_date_interval

        #If the ending value of the interval is a float or int then the user has moved the range
        if(type(range_tool.x_range.end) == float or
           type(range_tool.x_range.end) == int):
            
            #Extract the end value of the interval and convert it into correct format for datepicker
            new_end_date = datetime.fromtimestamp(range_tool.x_range.end / 1e3).date()
            
            #Check whether the date is less than the maximum of of the data
            if(new_end_date <= max_date_interval):
                #If the new end date is before the global maximum
                #Set the end date to the new end date
                end_date_input.value = new_end_date
            else:
                #If the value is after the global maximum
                #then set the value to the global maximum
                end_date_input.value = max_date_interval

    #Callback for when the datepicker of starting date changes
    #AND button is pressed (due to asynchronus behavior of udpates)
    def date_picker_to_range_tool_handler(event):
        
        start = start_date_input.value
        end = end_date_input.value
        
        range_tool.x_range.start = datetime.fromisoformat(start).timestamp() * 1e3
        range_tool.x_range.end = datetime.fromisoformat(end).timestamp() * 1e3
                
        IDs = time_table[start:end]['ID'].values
        
        #Compute the new points on the map
        new_map_data = compute_map_data(IDs)
        
        color_mapper = log_cmap(field_name = 'density', palette = palette,
                                low = new_map_data['density'].min(),
                                high = new_map_data['density'].max())
        
        color_bar.color_mapper=color_mapper['transform']
        
        source_map.data = new_map_data
        
        new_hist_data = compute_vbar_hist(IDs)
        
        source_hist.data = new_hist_data
        
        new_diff_data = measure_diff_hist(new_hist_data)
        
        source_diff.data = new_diff_data
        
        
    
    update_time_button.on_click(date_picker_to_range_tool_handler)
    
    #Set the global max and min for the date ranges
    min_date_interval = TMH_data.iloc[0]['created_at_CET'].date()
    max_date_interval = TMH_data.iloc[-1]['created_at_CET'].date()
    
    #Create a datepicker for the start date
    start_date_input = DatePicker(title='Start date', value=min_date_interval,
                            min_date=min_date_interval, max_date=max_date_interval,
                            width = int(screen_length*0.1),
                            height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #start_date_input.on_change('value',date_picker_to_range_tool_handler_start)
    
    #Create a datepicker for the end date
    end_date_input = DatePicker(title='End date', value=max_date_interval,
                          min_date=min_date_interval, max_date=max_date_interval,
                          width = int(screen_length*0.1),
                          height = int(screen_height*0.075))
    
    #Link the datepicker to the callback for when the value changes
    #end_date_input.on_change('value',date_picker_to_range_tool_handler_end)
    
    #Declare a RangeTool for the selection of the date ranges
    range_tool = RangeTool(x_range=Range1d(start=TMH_data.iloc[0]['created_at_CET'],
                                           end=TMH_data.iloc[-1]['created_at_CET']))
    range_tool.overlay.fill_color = "navy"
    range_tool.overlay.fill_alpha = 0.1
    
    #Link the range_tool to the callbacks for its interval
    range_tool.x_range.on_change('start',range_tool_to_datepicker_handler)
    range_tool.x_range.on_change('end',range_tool_to_datepicker_handler)
    
    #Add the range_tool to the Timeline plot
    Timeline.add_tools(range_tool)
    Timeline.toolbar.active_multi = range_tool

    ###  Timeline (END) ###
    
    ### Map (START) ###
    
    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON_RETINA)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    geo_map = figure(x_range=(650000, 1180000), y_range=(5700000, 6100000),
                     x_axis_type="mercator", y_axis_type="mercator",
                     plot_width=int(screen_length*0.5),
                     plot_height=int(screen_height*0.7))
    
    #Add the selector tools Lasso and Boxselect
    geo_map.add_tools(LassoSelectTool())
    geo_map.add_tools(BoxSelectTool())
    
    #Add the background to the map
    geo_map.add_tile(tile_provider)
    
    #Compute the data for the map
    map_data = compute_map_data()
    
    #Load in the map data
    source_map = ColumnDataSource(map_data)
    
    #Initialize the color mapper based on the values of density in the map_datat
    color_mapper = log_cmap(field_name = 'density', palette = palette,
                               low = map_data['density'].min(),
                               high = map_data['density'].max())
    
    #Defines color bar which indicates the levels
    color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     ticker = LogTicker(num_minor_ticks=1), 
                     label_standoff = 13, width=20, location=(0,0))# Set color_bar location
    
    #Add the color bar to the figure
    geo_map.add_layout(color_bar, 'left')
    
    #Plot the data points with their color corresponding to the amount of tweets at the location
    geo_map.circle(x='x_mercator', y='y_mercator', size=2, color=color_mapper, alpha=0.5, source=source_map)
    
    ### Map (END) ###
    
    ### MAP TO TIMELINE (START) ###
    
    #Declare a button which will update the values timeline according to the selected points in the map
    update_geo_button = Button(label="Sync Geo", button_type="success",
                                width = int(screen_length*0.1),
                                height = int(screen_height*0.075))
    
    def geo_to_timeline_handler(event):
        #print(map_data.iloc[source_map.selected.indices])
        #print(start_date_input.value,end_date_input.value)
        print(source_hist.data)
    
    update_geo_button.on_click(geo_to_timeline_handler)
    
    ### MAP TO TIMELINE (END) ###
    
    ###  Histogram (START) ###
    
    measures = ['Overall.score','Anger','Confusion','Disgust','Fear','Happiness','Sadness','Shame','Surprise']
    
    hist_data = compute_vbar_hist()
    
    source_hist = ColumnDataSource(hist_data)
    
    histogram = figure(plot_width=int(screen_length*0.5),plot_height=int(screen_height*0.7),
                  x_range=FactorRange(*hist_data['measure'].values),toolbar_location=None, title="Scores per Category",
                      y_axis_label="Score")
    
    histogram.vbar(x='measure', top='mean', width=1, source=source_hist,
                  fill_color=factor_cmap('measure', palette=OrRd3, factors=["Selected","Total"], start=1, end = 2))
    
    histogram.y_range.start = 0
    histogram.x_range.range_padding = 0.1
    histogram.xaxis.major_label_orientation = 1
    histogram.xgrid.grid_line_color = None
    
    tab_hist = Panel(child=histogram, title="Histogram")
    
    ###  Histogram (END) ###
    
    ### Differences (START) ###
    
    differences = figure(plot_width=int(screen_length*0.5),plot_height=int(screen_height*0.7),
                         x_range=measures,toolbar_location=None,y_axis_label="Percentage difference from overall")
    
    diff_data = measure_diff_hist(hist_data)
    
    source_diff = ColumnDataSource(diff_data)
    
    differences.vbar(x='measure',top='difference', source=source_diff,width=0.9,fill_color=OrRd3[0])
    
    tab_diff = Panel(child=differences, title="Differences")
    
    ### Differences (END) ###
    

    plots = layout([
        [Timeline,[start_date_input,end_date_input,update_time_button]],
        [geo_map,Tabs(tabs=[tab_hist,tab_diff])],
        
    ])
    doc.add_root(plots)

In [15]:
show(TMH_app)